##PJ : Iqbal Naufal - 13518074

Installing Library

In [ ]:
pip install textblob

Mouting Google Drive To use the necessary datasets

Note: I use it because my google colab doesn't let me upload the necessary datasets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Importing Libraries

In [3]:
from textblob import TextBlob
from textblob.en import Spelling
from collections import Counter
import gensim
import re

Compare Function to compare 2 text

In [4]:
def compare(text1, text2):  
    l1 = text1.split()
    l2 = text2.split()
    same = 0
    diff = 0
    for i in range(0, len(l1)):
        if l1[i] != l2[i]:
            diff += 1
        else:
            same += 1
    return (same, diff)

Correction Function using library TextBlob

In [9]:
def blobCorrect(text):
  txtBlb = TextBlob(text)
  corrected = txtBlb.correct()
  return corrected

Adding text with typo and correcting it

In [14]:
text = "Aoccdrnig to a rscheearch at Cmabrigde Uinervtisy, it deosn’t matter in what order the letters in a word are the only iprmoetnt thing is that the first and last later be at the right pclae. The set can be a total mess and you can still red it outfit porbelm. His is bcuseae the human mind does not red even letter by istlef, but the word as a wlohe."
textCorrected = blobCorrect(text)

Adding proper text

In [16]:
proper_text = "According to a research at Cambridge University, it doesn't matter in what order the letters in a word are, the only important thing is that the first and last letter be at the right place. The rest can be a total mess and you can still read it without problem. This is because the human mind does not read every letter by itself, but the word as a whole."

Calculating accuracy

In [17]:
difference = compare(textCorrected, proper_text)
acc = difference[0] / (difference[0] + difference[1]) * 100
print("Accuracy: ", acc, "%")

Accuracy:  78.26086956521739 %


Adding dataset to train library

In [18]:
with open("../Dataset/train.txt", "r") as f2:
    train = f2.read()
    lower_train = train.lower()

words = re.findall("[a-z]+", lower_train)
vocab = set(words)

onestring = " ".join(words)

path = "res.txt"
spelling = Spelling(path = path)
spelling.train(onestring, path)

Correcting the typo word

In [19]:
kalimat = text.split()
corrected = ""
for i in kalimat :
    corrected = corrected +" "+ spelling.suggest(i)[0][0]
print(corrected)

 Aoccdrnig to a rscheearch at Cmabrigde Uinervtisy, it deosn’t matter in what order the letters in a word are the only iprmoetnt thing is that the first and last later be at the right pclae. The set can be a total less and you can still red it unfit porbelm. His is bcuseae the human mind does not red even letter by istlef, but the word as a wlohe.


Calculating accuracy

In [21]:
difference = compare(corrected, proper_text)
acc = difference[0] / (difference[0] + difference[1]) * 100
print("Accuracy: ",acc, "%")

Accuracy:  71.01449275362319 %


Importing libraries for deep learning

In [52]:
import string
import re
import numpy as np
from numpy import array, argmax, random, take
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from tensorflow.keras import optimizers

Adding dataset

In [53]:
f3 = open("../Dataset/aspell.txt", mode='rt', encoding='utf-8')
text = f3.read()
f3.close()

Splitting labels

In [54]:
vocab = text.strip().split('\n')
vocab = [i.split(': ') for i in vocab]
vocab = array(vocab)

Splitting multiple value on one label

In [55]:
print(len(vocab))
for i in range(len(vocab)):
    vocab[i,0] = vocab[i,0].lower()
    vocab[i,1] = vocab[i,1].lower()
    temp = vocab[i,1].split(" ")
    if (len(temp) > 1):
        corr = vocab[i,0]
        for j in temp:
            vocab = np.append(vocab, [[corr, j]], 0)
        vocab = np.delete(vocab, i, 0)
vocab

450


array([['nevada', 'nevade'],
       ['presbyterian', 'presbyterian'],
       ['rsx', 'rsx'],
       ...,
       ['unnatural', 'unatourral'],
       ['unnatural', 'unaturral'],
       ['unnatural', 'unnaturral']], dtype='<U122')

Tokenization function

In [56]:
def tokenization(lines):
      tokenizer = Tokenizer()
      tokenizer.fit_on_texts(lines)
      return tokenizer

Creating tokenizer for the correct label

In [57]:
corr_tokenizer = tokenization(vocab[:, 0])
corr_vocab_size = len(corr_tokenizer.word_index) + 1

corr_length = 8
print('Correct Vocabulary Size: %d' % corr_vocab_size)

print(corr_tokenizer)

Correct Vocabulary Size: 459


Creating tokenizer for the typo label

In [58]:
typo_tokenizer = tokenization(vocab[:, 1])
typo_vocab_size = len(typo_tokenizer.word_index) + 1

typo_length = 8
print('Typo Vocabulary Size: %d' % typo_vocab_size)

Typo Vocabulary Size: 530


Encoding the sequence

In [59]:
def encode_sequences(tokenizer, length, lines):
         seq = tokenizer.texts_to_sequences(lines)
         seq = pad_sequences(seq, maxlen=length, padding='post')
         return seq

Splitting dataset to train and test

In [60]:
from sklearn.model_selection import train_test_split

# split data into train and test set
train, test = train_test_split(vocab, test_size=0.2)

Creating X and Y label on train and test datasets

In [61]:
# prepare training data
trainX = encode_sequences(typo_tokenizer, typo_length, train[:, 1])
trainY = encode_sequences(corr_tokenizer, corr_length, train[:, 0])

# prepare validation data
testX = encode_sequences(typo_tokenizer, typo_length, test[:, 1])
testY = encode_sequences(corr_tokenizer, corr_length, test[:, 0])

Building LSTM Model

In [62]:
# build NMT model
def define_model(in_vocab, out_vocab, in_timesteps, out_timesteps, units):
      model = Sequential()
      model.add(Embedding(in_vocab, units, input_length=in_timesteps, mask_zero=True))
      model.add(LSTM(units))
      model.add(RepeatVector(out_timesteps))
      model.add(LSTM(units, return_sequences=True))
      model.add(Dense(out_vocab, activation='softmax'))
      return model

Compiling the model

In [63]:
# model compilation
model = define_model(typo_vocab_size, corr_vocab_size, typo_length, corr_length, 512)
rms = optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=rms, loss='sparse_categorical_crossentropy', metrics=["accuracy"])

Model Fitting

In [64]:
filename = 'model.h1'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# train model
history = model.fit(trainX, trainY.reshape(trainY.shape[0], trainY.shape[1], 1),
                    epochs=30, batch_size=512, validation_split = 0.2,callbacks=[checkpoint], 
                    verbose=1)

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 6.1289 - accuracy: 7.4405e-04
Epoch 00001: val_loss improved from inf to 6.06982, saving model to model.h1


INFO:tensorflow:Assets written to: model.h1/assets


INFO:tensorflow:Assets written to: model.h1/assets


1/1 [==============================] - 28s 28s/step - loss: 6.1289 - accuracy: 7.4405e-04 - val_loss: 6.0698 - val_accuracy: 0.8705
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 6.0549 - accuracy: 0.8724
Epoch 00002: val_loss improved from 6.06982 to 5.75432, saving model to model.h1


INFO:tensorflow:Assets written to: model.h1/assets


...

INFO:tensorflow:Assets written to: model.h1/assets


1/1 [==============================] - 19s 19s/step - loss: 0.7607 - accuracy: 0.8724 - val_loss: 2.0323 - val_accuracy: 0.8601
Epoch 22/30
1/1 [==============================] - ETA: 0s - loss: 0.7586 - accuracy: 0.8724
Epoch 00022: val_loss improved from 2.03228 to 2.03204, saving model to model.h1


INFO:tensorflow:Assets written to: model.h1/assets


INFO:tensorflow:Assets written to: model.h1/assets


1/1 [==============================] - 19s 19s/step - loss: 0.7586 - accuracy: 0.8724 - val_loss: 2.0320 - val_accuracy: 0.8601
Epoch 23/30
1/1 [==============================] - ETA: 0s - loss: 0.7564 - accuracy: 0.8724
Epoch 00023: val_loss did not improve from 2.03204
1/1 [==============================] - 0s 104ms/step - loss: 0.7564 - accuracy: 0.8724 - val_loss: 2.0321 - val_accuracy: 0.8601
Epoch 24/30
1/1 [==============================] - ETA: 0s - loss: 0.7542 - accuracy: 0.8724
Epoch 00024: val_loss did not improve from 2.03204
1/1 [==============================] - 0s 98ms/step - loss: 0.7542 - accuracy: 0.8724 - val_loss: 2.0325 - val_accuracy: 0.8601
Epoch 25/30
1/1 [==============================] - ETA: 0s - loss: 0.7520 - accuracy: 0.8724
Epoch 00025: val_loss did not improve from 2.03204
1/1 [==============================] - 0s 91ms/step - loss: 0.7520 - accuracy: 0.8724 - val_loss: 2.0332 - val_accuracy: 0.8601
Epoch 26/30
1/1 [==============================] - ETA:

Prediction using test dataset

In [65]:
model = load_model('model.h1')
predict_x= model.predict(testX.reshape((testX.shape[0],testX.shape[1]))) 
classes_x= argmax(predict_x,axis=1)
print(classes_x.shape)

(105, 459)


Output for labels

In [66]:
print(classes_x)

[[0 0 0 ... 0 0 0]
 [5 2 2 ... 1 1 1]
 [5 2 2 ... 1 2 1]
 ...
 [6 3 3 ... 2 2 2]
 [7 4 4 ... 3 3 3]
 [5 3 3 ... 2 2 2]]


Function for getting word from the tokenizer

In [67]:
def word_token(n, tokenizer):
      for word, index in tokenizer.word_index.items():
          if index == n:
              return word
      return None

Creating the prediction for test dataset

In [73]:
pred_text = []
for i in classes_x:
  max = 0
  idx = 0
  for j in range(len(i)):
    if (i[j] > max):
      max = i[j]
      idx = j
    t = word_token(idx, corr_tokenizer)
  pred_text.append(t)

Creating dataframe for the output

In [74]:
pred_df = pd.DataFrame({'actual' : test[:,0], 'predicted' : preds_text})

The output

In [75]:
pred_df.sample(15)

,actual,predicted
5,asymptote,beginning
78,want,beginning
7,situation,beginning
80,harass,bureaucracy
23,likely,beginning
24,directly,bureaucracy
65,orient,beginning
52,subscribed,beginning
96,signal,bureaucracy
46,occurrence,beginning


The output

Note: The prediction is still far off because of the size of the dataset that is far too small to use and the word frequencies on the dataset is not enough to feed to the model.

In [76]:
pred_df.tail(15)

,actual,predicted
90,pronunciation,beginning
91,john,beginning
92,maximum,butter
93,trouble,psychologist
94,what,beginning
95,about,beginning
96,signal,bureaucracy
97,simplicity,beginning
98,function,beginning
99,section,butter
